In [ ]:
import logging
import os
from pathlib import Path

import cm1.input.sounding
import pandas as pd
import xarray as xr
from cm1.run import PBS, CM1Run
from cm1.utils import animate_cm1out_nc, skewt
from metpy.units import units

In [ ]:
# Path to the CM1 repository
scratch_dir = os.getenv("SCRATCH", "/Users/ahijevyc/sysdisk1")
if scratch_dir is None:
	raise EnvironmentError("The SCRATCH environment variable is not set.")
cm1_path = Path(scratch_dir) / "CM1"

In [ ]:
testcase = "supercell"
run_dir = cm1_path / f"run_{testcase}"

pbs_account = os.getenv("PBS_ACCOUNT", "NMMM0001")
if pbs_account is None:
    raise EnvironmentError("The PBS_ACCOUNT environment variable is not set.")

pbs_config = PBS(
    name=testcase,
    account=pbs_account,
    walltime="01:00:00",
    nodes=1,
    run_dir=run_dir,
    executable_path=cm1_path / "run/cm1.exe",
)

# If the testcase is 1 or 2-D use cm1 compiled for single processor
oneortwod = testcase.startswith("scm_") or testcase == "nh_mountain_waves"
if oneortwod:
    pbs_config.executable_path = cm1_path / "run/cm1.single.gnu.exe"

In [ ]:
cm1_run = CM1Run(cm1_path=cm1_path, pbs_config=pbs_config)
# Modify values in namelist
namelist = cm1_run.namelist
# Combine output in one netCDF file.
namelist["param9"]["output_format"] = 2
namelist["param9"]["output_filetype"] = 1

input_sounding_ds = None
if testcase in ["supercell"]:
    valid_time = pd.to_datetime("20240525")
    lon = -95 * units.degree_E
    lat = 33 * units.degree_N
    if os.path.exists("/glade/campaign"):
        input_sounding_ds = cm1.input.sounding.era5_model_level(
            valid_time, lon=lon, lat=lat
        )
    else:
        input_sounding_ds = cm1.input.sounding.era5_aws(valid_time, lon=lon, lat=lat)

    ofile = run_dir / "input_sounding"
    if os.path.exists(ofile):
        old_sounding = open(ofile, "r").read()
        assert (
            old_sounding == input_sounding_ds.to_txt()
        ), f"{ofile} already exists and differs"
    else:
        with open(ofile, "w") as f:
            f.write(input_sounding_ds.to_txt())
            logging.warning(f"wrote {ofile}")
    cm1_run.sounding = input_sounding_ds

runlocal = True  # set True to run locally as on cmd line
if oneortwod or runlocal:
    cm1_run.run_serial(background=False)
else:
    cm1_run.submit_job()
print(cm1_run.readme)

In [ ]:
cm1_run.sounding.plot()

In [ ]:
cm1.input.era5.aws(valid_time)

In [ ]:
# Open NetCDF file
output_ds = xr.open_dataset(run_dir / "cm1out.nc", decode_timedelta=True)

animate_cm1out_nc(output_ds, "dbz", height=0.1, dim="zh", vmin=-16, vmax=60, cmap="gist_ncar")

In [ ]:
help(CM1Run)